In [ ]:
#
# Prototyping for the following ... now it's about optimization
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from shapely import Polygon
from   udist_scatterplots_via_sectors          import UDistScatterPlotsViaSectors
from   udist_scatterplots_via_sectors_tile_opt import UDistScatterPlotsViaSectorsTileOpt
import time
import rtsvg
rt = rtsvg.RACETrack()
import random
num_of_pts   = [100, 200, 50]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

_iterations_ = 2

t1 = time.time()
udspvs          = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_, debug=True)
t2 = time.time()
udspvs_tile_opt = UDistScatterPlotsViaSectorsTileOpt(_xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_, debug=True, num_of_tiles=32)
t3 = time.time()
# 351 Points | 512 Iterations | 5.66s Polars Time | 10.68s Polars Tile Opt Time | (M1 Pro 16G) 
print(f'{len(_xvals_)} Points | {_iterations_} Iterations | {t2-t1:.2f}s Polars Time | {t3-t2:.2f}s Polars Tile Opt Time | (M1 Pro 16G) ')
#rt.tile([udspvs.animateIterations(animation_dur="4s"), udspvs_tile_opt.animateIterations(animation_dur="4s")], spacer=10)

In [ ]:
self = udspvs_tile_opt
w_step, h_step = 256, 256
pt_i, iter = 15, 0
_tiles_ = []

_df_ = self.df_tile_determinations[iter]
svg = [f'<svg x="0" y="0" width="{w_step}" height="{h_step}" viewBox="0.0 0.0 1.0 1.0" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="0" y="0" width="1.0" height="1.0" x="0" y="0" fill="#ffffff" />')
_xiyis_ = set()
for i in range(len(_df_)):
    xi, yi      = _df_['xi'][i], _df_['yi'][i]
    _xiyis_.add((xi,yi))
for _xy_ in _xiyis_:
    xi, yi = _xy_
    x0,y0,x1,y1 = self.tileBounds(xi,yi)
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" x="0" y="0" fill="none" stroke="#000000" stroke-width="0.001"/>')
for i in range(len(_df_)):
    x, y, w, c = _df_['x'][i], _df_['y'][i], _df_['w'][i], _df_['c'][i]
    svg.append(f'<circle cx="{x}" cy="{y}" r="0.003" fill="{c}" />')
_df_ = _df_.filter(pl.col('__index__') == pt_i)

svg.append('</svg>')

_tiles_.append(''.join(svg))

rt.tile(_tiles_)